# Merging Dataframes


4 types of merging, merge with index

In [ ]:
Gross_df = pd.DataFrame([{'Title': 'Deadpool', 'Gross': 783.11},
                         {'Title': 'Captain America', 'Gross': 1153.30},
                         {'Title': 'Ip Man 3', 'Gross': 156.13}])
Gross_df = Gross_df.set_index('Title')
Distributor_df = pd.DataFrame([{'Title': 'Deadpool', 'Distributor': 'Fox'},
                         {'Title': 'Captain America', 'Distributor': 'Disney'},
                         {'Title': '踏血尋梅', 'Distributor': '美亞'}])
Distributor_df = Distributor_df.set_index('Title')
print(Gross_df.head())
print()
print(Distributor_df.head())

In [ ]:
pd.merge(Gross_df, Distributor_df, how='outer', left_index=True, right_index=True)

In [ ]:
pd.merge(Gross_df, Distributor_df, how='inner', left_index=True, right_index=True)

In [ ]:
pd.merge(Gross_df, Distributor_df, how='left', left_index=True, right_index=True)

In [ ]:
pd.merge(Gross_df, Distributor_df, how='right', left_index=True, right_index=True)

Merge with column name

In [ ]:
Gross_df = Gross_df.reset_index()
Distributor_df = Distributor_df.reset_index()
pd.merge(Gross_df, Distributor_df, how='left', left_on='Title', right_on='Title')

Columns with same name

In [ ]:
movie_df = pd.DataFrame([{'Title': 'Deadpool', 'Distributor': 'Fox', 'Gross': 783.11},
                   {'Title': 'Captain America: Civil War', 'Distributor': 'Disney', 'Gross': 1153.30},
                   {'Title': 'Batman v Superman: Dawn of Justice', 'Distributor': 'Warner Bros.', 'Gross': 873.63}],
                  columns=['Title', 'Distributor', 'Gross']
                 )
distributor_bf = pd.DataFrame([{'Distributor': 'Fox', 'Gross': 13280},
                   {'Distributor': 'Disney', 'Gross': 55137},
                   {'Distributor': 'Warner Bros.', 'Gross': 12992}],
                  columns=['Distributor', 'Gross']
                 )

pd.merge(movie_df, distributor_bf, how='left', left_on='Distributor', right_on='Distributor')

# Exercise 1

Please filter df_movies having the above 3 movies, and merge with df_ratings.

In [ ]:
import pandas as pd

df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')
print(df_movies.info())
print(df_ratings.info())

movies = '|'.join(movie_df['Title'])

# ...

df_sub_movies = df_movies[df_movies['title'].str.contains(movies)]
df_sub = pd.merge(df_ratings, df_sub_movies, how='inner', left_on='movieId', right_on='movieId')
df_sub

# Group by

In [ ]:
import pandas as pd
import numpy as np

df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

df = pd.merge(df_ratings, df_movies, how='inner', left_on='movieId', right_on='movieId')
df.info()

In [ ]:
%%timeit -n 1
for title in df['title'].unique():
    avg = np.average(df.where(df['title']==title).dropna()['rating'])
    print('Movie ' + title + ' have an average rating of ' + str(avg))

In [ ]:
%%timeit -n 1
for title, frame in df.groupby('title'):
    avg = np.average(frame['rating'])
    print('Movie ' + title + ' have an average rating of ' + str(avg))

In [ ]:
df.head()

In [ ]:
df.groupby('title').agg({'rating': np.average})

In [ ]:
print(type(df.groupby('title')['rating'].agg({'avg': np.average})))
print(type(df.groupby('title')['rating'].agg({'avg': np.average, 'count': pd.Series.count})))

In [ ]:
pd.set_option('max_rows', 10)

In [ ]:
(df.set_index('title').groupby(level=0)['rating']
    .agg({'avg': np.average, 'sum': np.sum}))

In [ ]:
(df.set_index('title').groupby(level=0)['rating', 'timestamp']
    .agg({'max': np.max, 'min': np.min}))

# Exercise 2

Please transform the timestamp variable to find no. of ratings in each year.

In [ ]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df['year'] = df['datetime'].dt.year

# ...

df.groupby('year').agg({'rating': {'avg': np.average, 'count': pd.Series.count}})

# Scales

In [ ]:
df = pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                  index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 'ok', 'ok', 'ok', 'poor', 'poor'])
df.rename(columns={0: 'Grades'}, inplace=True)
df

In [ ]:
df['Grades'].astype('category').head()

In [ ]:
grades = df['Grades'].astype('category',
                             categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'],
                             ordered=True)
grades.head()

In [ ]:
grades > 'C'

In [ ]:
import pandas as pd
import numpy as np

df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

df = pd.merge(df_ratings, df_movies, how='inner', left_on='movieId', right_on='movieId')

df = df.set_index('title').groupby(level=0)['rating'].agg({'avg': np.average})
pd.cut(df['avg'],10)

# Pivot Tables

In [ ]:
import pandas as pd
import numpy as np

df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

df = pd.merge(df_ratings, df_movies, how='inner', left_on='movieId', right_on='movieId')
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df['weekday'] = df['datetime'].dt.weekday

In [ ]:
df.head()

In [ ]:
df[df['datetime'].dt.year>2010] \
    .pivot_table(values='rating', index='title', columns='weekday', aggfunc=np.mean) \
    .dropna()

In [ ]:
df[df['datetime'].dt.year>2010] \
    .pivot_table(values='rating', index='title', columns='weekday', aggfunc=np.mean, margins=True) \
    .dropna()

# Exercise 3

In [ ]:
import pandas as pd

df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

movie_df = pd.DataFrame([{'Title': 'Deadpool', 'Distributor': 'Fox', 'Gross': 783.11},
                   {'Title': 'Captain America: Civil War', 'Distributor': 'Disney', 'Gross': 1153.30},
                   {'Title': 'Batman v Superman: Dawn of Justice', 'Distributor': 'Warner Bros.', 'Gross': 873.63}],
                  columns=['Title', 'Distributor', 'Gross']
                 )
movies = '|'.join(movie_df['Title'])

df_sub_movies = df_movies[df_movies['title'].str.contains(movies)]
df_sub = pd.merge(df_ratings, df_sub_movies, how='inner', left_on='movieId', right_on='movieId')
df_sub['datetime'] = pd.to_datetime(df_sub['timestamp'], unit='s')
df_sub['weekday'] = df_sub['datetime'].dt.weekday
df_sub

Please return a pivot table - showing count of rating against weekday and rating values.

In [ ]:
# ...

df_sub.pivot_table(values='userId', index='title', columns='rating', aggfunc='count', margins=True)

# Date Functionality in Pandas

In [ ]:
import pandas as pd
import numpy as np

### Timestamp

In [ ]:
pd.Timestamp('9/1/2016 10:05AM')

### Period

In [ ]:
pd.Period('1/2016')

In [ ]:
pd.Period('3/5/2016')

### DatetimeIndex

In [ ]:
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), pd.Timestamp('2016-09-03')])
t1

In [ ]:
type(t1.index)

### PeriodIndex

In [ ]:
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), pd.Period('2016-11')])
t2

In [ ]:
type(t2.index)

### Converting to Datetime

In [ ]:
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']
ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index=d1, columns=list('ab'))
ts3

In [ ]:
ts3.index = pd.to_datetime(ts3.index)
ts3

In [ ]:
pd.to_datetime('4.7.12', dayfirst=True)

### Timedeltas

In [ ]:
pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016')

In [ ]:
pd.Timestamp('9/2/2016 8:10AM') + pd.Timedelta('12D 3H')

### Working with Dates in a Dataframe

In [ ]:
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
dates

In [ ]:
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 9).cumsum(),
                  'Count 2': 120 + np.random.randint(-5, 10, 9)}, index=dates)
df

In [ ]:
df.index.weekday_name

In [ ]:
df.diff()

In [ ]:
df.resample('M').mean()

In [ ]:
df['2017']

In [ ]:
df['2016-12']

In [ ]:
df['2016-12':]

In [ ]:
df.asfreq('W', method='ffill')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df.plot()